https://code.activestate.com/recipes/578625-python-ast-to-xml/

or try this

https://github.com/flyte/ast-to-xml/blob/develop/ast_to_xml.py

or

https://github.com/hchasestevens/astpath/blob/master/astpath/asts.py#L30

or

https://www.reddit.com/r/Python/comments/79k5ve/is_there_any_way_to_round_trip_a_python_ast/


In [5]:
import ast, sys
from xml.dom import minidom

from xml.etree import cElementTree as etree


def prettify(xml_string):
    reparsed = minidom.parseString(xml_string)
    return reparsed.toprettyxml(indent="  ") 

class Ast2xml(ast.NodeVisitor):
    def __init__(self):
        super(ast.NodeVisitor, self).__init__()
        self.path = []
        self.root = etree.Element('ast')
        self.celement = self.root
        self.lname = 'module'

    def convert(self, tree):
        self.visit(tree)
        return etree.tostring(self.root)
        
    def generic_visit(self, node):
        self.path.append(type(node).__name__)
        ocelement = self.celement
        self.celement = etree.SubElement(self.celement, self.lname)
        self.celement.attrib.update({'_name': type(node).__name__})
        olname = self.lname
        self.lname = type(node).__name__
        for item in node.__dict__:
            self.lname = item
            if isinstance(getattr(node, item), ast.AST):
                self.generic_visit(getattr(node, item))
            elif isinstance(getattr(node, item), list):
                ocel2 = self.celement
                olname2 = self.lname
                self.celement = etree.SubElement(self.celement, self.lname)
                self.celement.attrib.update({'_name': '_list'})
                self.lname = '_list_element'
                [self.generic_visit(childnode) for childnode in getattr(node, item) if isinstance(childnode, (ast.AST, list))]
                self.celement = ocel2
                self.lname = olname2
            else:
                self.celement.attrib.update({item: str(getattr(node, item))})
        self.path.pop()
        self.celement = ocelement
        self.lname = olname


In [7]:
test = "print('hello world')"
astree = ast.parse(test)
res = Ast2xml().convert(astree)

RecursionError: maximum recursion depth exceeded while calling a Python object